In [98]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools import cache
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest, batched, pairwise
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx
inf = float('inf')

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, complexneighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot, animate
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

In [99]:
f = open("in.txt").read().split("\n")
lines = [list(line) for line in f]
grid, neigh = gridneigh("in.txt", to_dict=True, parser=lambda x: [(ch) for ch in x])


In [14]:
def neigh(n):
    if grid[n] == '|':
        return [n+complex(0,1), n + complex(0,-1)]
    elif grid[n] == '-':
        return [n+complex(1,0), n + complex(-1,0)]
    elif grid[n] == 'L':
        return [n+complex(0,1), n + complex(1,0)]
    elif grid[n] == 'J':
        return [n+complex(0,1), n + complex(-1,0)]
    elif grid[n] == '7':
        return [n+complex(0,-1), n + complex(-1,0)]
    elif grid[n] == 'F':
        return [n+complex(0,-1), n + complex(1,0)]
    elif grid[n] == '.':
        print('error')
    elif grid[n] == 'S':
        return [n+complex(0,1), n+complex(0,-1)]
        # return [n+i for i in [complex(1,0), complex(-1,0), complex(0,1), complex(0,-1)] if grid.get(n+i,'.') != '.']
    else:
        print('error')
        
def neigh2(node):
    global safe, distances, sq
    res = set()
    n = node
    if node.real.is_integer() and node.imag.is_integer():
        # in grid
        for i in [complex(1,0), complex(-1,0), complex(0,1), complex(0,-1)]:
            if (node + i) in grid and (node+i) not in distances:
                res.add(node+i)
        for i in [complex(0.5,1), complex(-0.5,1), complex(0.5,-1), complex(-0.5,-1)]:
            # print(node+i,grid.get(node+i, 'x'))
            if grid.get(node+i, 'x') == '|':
                res.add(node+i)
        for i in [complex(1,0.5), complex(1,-0.5), complex(-1,0.5), complex(-1,-0.5)]:
            if grid.get(node+i, 'x') == '-':
                res.add(node+i)
    elif grid[node] == '|':
        # were in sq
        for i in [complex(0,1), complex(0,-1)]:
            if grid.get(node+i, 'x') == '|':
                res.add(node+i)
        for i in [complex(0.5,0.5), complex(-0.5,-0.5), complex(-0.5,0.5), complex(0.5,-0.5)]:
            if grid.get(node+i, 'x') == '-':
                res.add(node+i)
                
        for i in [complex(0.5,1), complex(-0.5,1), complex(0.5,-1), complex(-0.5,-1)]:
            if (node + i) in grid and (node+i) not in distances:
                res.add(node+i)
 
        
    elif grid[node] == '-':
        for i in [complex(1,0), complex(-1,0)]:
            if grid.get(node+i, 'x') == '-':
                res.add(node+i)
                
        for i in [complex(0.5,0.5), complex(-0.5,-0.5), complex(-0.5,0.5), complex(0.5,-0.5)]:
            if grid.get(node+i, 'x') == '|':
                res.add(node+i)
        for i in [complex(1,0.5), complex(1,-0.5), complex(-1,0.5), complex(-1,-0.5)]:
            if (node + i) in grid and (node+i) not in distances:
                res.add(node+i)
        
    return res
# test = complex(5,-8)
# grid[test+1j], neigh2(complex(5,-8))

In [95]:
# parse grid with complex numbers
f = open("t0.txt").read().split("\n")
lines = [list(line) for line in f]
lines = np.pad(lines,1)
grid = arr_to_dict(lines, tocomplex=True)

# E = 1,0
# W = -1,0
# N = 0,1
# S = 0,-1    
safe = set()
safeground = set()   
sq = {}
tiles = len(grid)

distances = {}
start = [k for k,v in grid.items() if v == 'S'].pop()
start
grid[start] = '|'
# t1
grid[start] = 'F'
# t0
grid[start] = '7'
border = set()
def bfs(node):
    dis = 0
    tocheck = deque([node])
    seen = set()
    while tocheck:
        for _ in range(len(tocheck)):
            cur = tocheck.popleft()
            # print(len(neigh2(cur)))
            for n in neigh(cur):
                if n not in seen:
                    seen.add(n)
                    distances[n] = dis + 1
                    tocheck.append(n)
        dis += 1
bfs(start)


loop = len(distances)        
def bfs(node):
    dis = 0
    tocheck = deque([node])
    seen = set()
    while tocheck:
        for _ in range(len(tocheck)):
            cur = tocheck.popleft()
            # print(len(neigh2(cur)))
            for n in neigh2(cur):
                if n not in seen:
                    if n.real.is_integer() and n.imag.is_integer():
                        safe.add(n)
                    if grid[n] == '.':
                        safeground.add(n)
                    seen.add(n)
                    tocheck.append(n)
        dis += 1
[(k,v) for k,v in distances.items() if v == max(distances.values())]
print(len(grid), len(distances), max(distances.values()))

for k,v in grid.items():
    if grid[k] != '0' and k not in distances:
        grid[k] = '.'
        
real = len(lines[0])
imag = len(lines)
for r in range(real):
    for i in range(0,-imag,-1):
        # print(r,c)
        if (grid.get(complex(r,i), 'x')) == 'x':
            print('error cannot be found')
            sys.exit()
            

        if grid.get(complex(r,i), 'x') not in list('-FL') and grid.get(complex(r+1,i),'x') not in list('7J-') and (complex(r,i) in distances or complex(r+1,i) in distances):
            # pipe N S
            sq[complex(r+0.5, i)] = '|'
        if grid.get(complex(r,i), 'x') not in list('7F|') and grid.get(complex(r,i-1),'x') not in list('LJ|')  and (complex(r,i) in distances or complex(r,i-1) in distances):
            # pipe WE
            sq[complex(r,i-0.5)] = '-'
        # if grid.get(complex(r,i), 'x') in list('|7J') and grid.get(complex(r+1,i),'x') in list('FL|'):
        #     # pipe N S
        #     sq[complex(r+0.5, i)] = '|'
        # if grid.get(complex(r,i), 'x') in list('-LJ') and grid.get(complex(r,i-1),'x') in list('F-7'):
        #     # pipe WE
        #     sq[complex(r,i-0.5)] = '-'

grid |= sq

        
for node in grid:
    if grid[node] == '0':
        border.add(node)
for node in border:
    bfs(node)


264 160 80


In [96]:
ground = sum(1 for k,v in grid.items() if v == '.')
ground, len(safeground), ground - len(safeground)

(40, 30, 10)

In [97]:
left = (set(grid) - set(sq) - set(distances) - set(safe) - border)
len(left)

10

In [84]:
len(grid), len(sq), len(distances), len(safe)

(32133, 11969, 13198, 6445)

In [329]:
tiles, loop, b, len(safe), tiles - loop - b - len(safe)

(264, 160, 564, 30, -490)

In [179]:
[(k,v) for k,v in distances.items() if v == max(distances.values())]
len(distances)

13198

In [ ]:
def neigh3(n):
    if grid[n] == '|':
        return [n+complex(0,1), n + complex(0,-1)]
    elif grid[n] == '-':
        return [n+complex(1,0), n + complex(-1,0)]
    elif grid[n] == 'L':
        return [n+complex(0,1), n + complex(1,0)]
    elif grid[n] == 'J':
        return [n+complex(0,1), n + complex(-1,0)]
    elif grid[n] == '7':
        return [n+complex(0,-1), n + complex(-1,0)]
    elif grid[n] == 'F':
        return [n+complex(0,-1), n + complex(1,0)]
    elif grid[n] == '.':
        print('error')
    elif grid[n] == 'S':
        return [n+complex(0,1), n+complex(0,-1)]
        # return [n+i for i in [complex(1,0), complex(-1,0), complex(0,1), complex(0,-1)] if grid.get(n+i,'.') != '.']
    else:
        print('error')

In [ ]:
def check(node):
    tocheck = [node]
    seen = set([node])
    while tocheck:
        cur = tocheck.pop()
        # print(len(neigh2(cur)))
        for n in neigh3(cur):
            if n not in seen:
                seen.add(n)
                tocheck.append(n)
                


In [120]:
ground, len(safe)

(31, 0)